To obtain the data with the rent prices, we will do web-scrapping from Idealista, a real estate platform that operates primarily in Spain. By scraping data from Idealista, we can access up-to-date information on rental prices across different regions.

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extraer_precios_ciudad(url, ciudad):
    """
    Extrae los precios históricos de alquiler por m² para una ciudad desde Idealista.
    
    Parámetros:
        url (str): URL de la ciudad en Idealista.
        ciudad (str): Nombre de la ciudad (para incluir en el DataFrame).

    Retorna:
        pandas.DataFrame: con columnas ['fecha', 'precio', 'ciudad']
    """
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept-Language": "es-ES,es;q=0.9",
    "Referer": "https://www.google.com/",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Connection": "keep-alive"
}

    
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Error al acceder a la página: {response.status_code}")
    
    soup = BeautifulSoup(response.content, "html.parser")
    rows = soup.select("tr.table__row")

    datos = []
    for row in rows:
        cells = row.find_all("td")
        if len(cells) >= 2:
            fecha = cells[0].get_text(strip=True)
            precio_valor = float(cells[1]['data-sortable'].replace(',', '.'))
            datos.append({
                "fecha": fecha,
                "precio": precio_valor,
                "ciudad": ciudad
            })

    return pd.DataFrame(datos)


In [4]:
url = "https://www.idealista.com/sala-de-prensa/informes-precio-vivienda/alquiler/cataluna/barcelona-provincia/barcelona/historico/"
df_barcelona = extraer_precios_ciudad(url, "Barcelona")

print(df_barcelona.head())


Exception: Error al acceder a la página: 403

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

def extraer_precios_con_selenium(url, ciudad):
    """
    Extrae precios de alquiler por m² desde una URL de Idealista usando Selenium.
    Muestra el navegador y permite inspección visual si algo falla.
    """
    options = Options()
    # No usamos modo headless para poder ver lo que ve el navegador
    # options.add_argument("--headless")  # Comentado para depurar

    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "tr.table__row"))
        )
    except:
        print("⚠️ No se encontró la tabla dentro del tiempo de espera.")
        input("👁 Pulsa Enter para cerrar el navegador...")
        driver.quit()
        return pd.DataFrame(columns=["fecha", "precio", "ciudad"])

    filas = driver.find_elements(By.CSS_SELECTOR, "tr.table__row")
    print(f"✅ Filas encontradas: {len(filas)}")

    datos = []

    for fila in filas:
        celdas = fila.find_elements(By.TAG_NAME, "td")
        if len(celdas) >= 2:
            fecha = celdas[0].text.strip()
            precio_texto = celdas[1].text.strip().replace(" €/m2", "").replace(",", ".")
            try:
                precio = float(precio_texto)
                datos.append({
                    "fecha": fecha,
                    "precio": precio,
                    "ciudad": ciudad
                })
            except ValueError:
                continue

    input("👁 Pulsa Enter para cerrar el navegador...")
    driver.quit()
    return pd.DataFrame(datos)


In [10]:
url = "https://www.idealista.com/sala-de-prensa/informes-precio-vivienda/alquiler/cataluna/barcelona-provincia/barcelona/historico/"
df = extraer_precios_con_selenium(url, "Barcelona")
print(df.head())


⚠️ No se encontró la tabla dentro del tiempo de espera.
Empty DataFrame
Columns: [fecha, precio, ciudad]
Index: []
